<a href="https://colab.research.google.com/github/julmiha25-sys/Python/blob/main/Python_%D0%98%D1%82%D0%BE%D0%B3%D0%BE%D0%B2%D1%8B%D0%B9_%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import smtplib # Для отправки электронной почты по протоколу SMTP
import ssl # Для безопасной передачи данных между скриптом и SMTP-сервером
from email.message import EmailMessage # Для создания и представления письма
import gspread  # Для работы с Google Sheets
from oauth2client.service_account import ServiceAccountCredentials as SAC  # Для авторизации в Google API
import requests, os
from datetime import datetime
import ast # Для работы с абстрактными синтаксическими деревьями
import logging # Реализация логирования
from pathlib import Path
import psycopg2 # Работа с Postgres
from dotenv import load_dotenv # Импорт функции для загрузки переменных окружения из файла .env

load_dotenv('1.env')  # Загружаем переменные из .env


# Авторизация и путь к JSON ключу
client = gspread.authorize(SAC.from_json_keyfile_name(GOOGLE_CREDS_FILE, [
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive'
]))

# Функция для отправки почты
def mail_message(sh_url):
  try:
    context=ssl.create_default_context() # Cоздание защищенного соединения SSL
    msg = EmailMessage() # Новый объект письма
    message=f"Скрипт на загрузку из API в БД Postgres отработал {sh_url}"
    msg.set_content(message)
    msg['Subject']='Оповещение о запуске скрипта'
    msg['From']='julmiha25@mail.ru'
    msg['To']='julmiha25@mail.ru'
    server=smtplib.SMTP_SSL('smtp.mail.ru', 465, context=context)  # Отправка письма
    server.login('julmiha25@mail.ru', EMAIL_PASSWORD)
    server.send_message(msg)
    server.quit()  # Закрытие соединения
    logging.info("Письмо успешно отправлено")
  except Exception as e:
    logging.error(f"Ошибка при отправке письма: {e}")

# Функция настройки логирования
def api_logging(prefix="api", keep=3):
  log_dir=Path.cwd() # Текущая директория /content
  logs=sorted(log_dir.glob(f"{prefix}_*.log"), key=os.path.getmtime)[:-(keep - 1) or None] # Сортировка и выборка самых старых файлов, кроме 2-х последних
  for log in logs: # Удаление старых файлов
    log.unlink()
  timestamp=datetime.now().strftime("%Y-%m-%d_%H:%M")
  log_file=log_dir/f"{prefix}_{timestamp}.log" # Определение имени log-файла
  logging.basicConfig(filename=str(log_file),level=logging.INFO,format='%(asctime)s - %(levelname)s - %(message)s',datefmt='%Y-%m-%d %H:%M:%S',force=True) # Настройки логирования
  return log_file
log_file=api_logging(prefix="api", keep=3) # Префикс файлф api, сохранять 3 последних лога

# Функция для получения и обработки данных с API
def get_post(url,params):
  try:
    URL = url
    r=requests.get(URL,params=params)  # Получение данных
    r.raise_for_status()  # Проверка HTTP статуса
    res=r.json()  # Десериализация
    logging.info(f"Начало скачивания данных")

  except Exception as e:  # Любая ошибка при получении данных
    logging.error(f"Ошибка доступа к API: {type(e).__name__} - {str(e)}")
    return []  # Возвращаем пустой список

  result=[] # Список со словарями для записи в БД

  for item in res: # Перебор элементов списка
    try:

      if not isinstance(item, dict): # Пропуск несловарей
        logging.error(f"Элемент не словарь: {type(item)}")
        continue

      try:
        params_str=item.get('passback_params') # Распаковка словаря passback_params в строку "{'key': 'value'}"
        if params_str is None: # Нет ключа passback_params
          logging.error(f"Нет ключа 'passback_params' в элементе: {item}")
          continue
        if not isinstance(params_str, str): # Элементы passback_params не строка
          logging.error(f"'passback_params' не строка: {item}")
          continue
        params_dict=ast.literal_eval(params_str) # Создание словаря с ключами-элементами passback_params

      except (SyntaxError, ValueError) as e:
        logging.error(f"Ошибка парсинга 'passback_params': {item}")
        continue

      # Создание словаря с ключами-полями и данными-строкой (1 строка для записи в БД) и валидация
      if ((item.get('lti_user_id') and isinstance(item.get('lti_user_id'),str) and
         (item.get('is_correct') in [None,0,1]) and
         (item.get('attempt_type') in ['submit','run']) and
         (item.get('created_at') and isinstance(item.get('created_at'),str)) and
         (params_dict.get('lis_result_sourcedid') is None or isinstance(params_dict.get('lis_result_sourcedid'),str)) and
         (params_dict.get('lis_outcome_service_url') is None or isinstance(params_dict.get('lis_outcome_service_url'),str)) and
         (params_dict.get('oauth_consumer_key') is None or isinstance(params_dict.get('oauth_consumer_key'),str)))):
         new_item={'lti_user_id': item.get('lti_user_id'),'oauth_consumer_key': params_dict.get('oauth_consumer_key', ''),
                    'lis_result_sourcedid': params_dict.get('lis_result_sourcedid', ''),
                    'lis_outcome_service_url': params_dict.get('lis_outcome_service_url', ''),
                    'is_correct': item.get('is_correct'),
                    'attempt_type': item.get('attempt_type'),
                    'created_at':datetime.strptime(item.get('created_at'), '%Y-%m-%d %H:%M:%S.%f')}
         result.append(new_item)

      else:
         logging.error(f"Элемент не прошел валидацию: {item}")

    except Exception:  # Прочая ошибка при обработке элемента
            logging.error(f"Ошибка данных: {item}")

  logging.info(f"Окончание скачивания данных")

  grouped_by_date={} # Словарь для группировкии по дате
  for item in result:
    date_key=item['created_at'].strftime('%Y-%m-%d')
    if date_key not in grouped_by_date:
      grouped_by_date[date_key]=[]
    grouped_by_date[date_key].append(item)
  #  Всего попыток за день
  cnt={date: len(items) for date, items in sorted(grouped_by_date.items())}
  #  Успешных попыток за день
  cnt_success={date: len([item for item in items if item.get('is_correct')==1]) for date, items in sorted(grouped_by_date.items())}
  #  Уникальных юзеров за день
  cnt_users={date: len(set(item['lti_user_id'] for item in items)) for date, items in sorted(grouped_by_date.items())}
  #  Среднее число решенных задач юзером за день
  mean_cnt={date: round(cnt_success[date]/cnt_users[date],2) for date in cnt_success.keys() if cnt_users[date]>0}

  print(f"Всего попыток за день {cnt}")
  print(f"Успешных попыток за день {cnt_success}")
  print(f"Количество уникальных юзеров за день {cnt_users}")
  print(f"Среднее количество решенных задач юзера за день {mean_cnt}")
  #upload(cnt, cnt_success, cnt_users, mean_cnt)
  sh_url=upload(cnt, cnt_success, cnt_users, mean_cnt)
  mail_message(sh_url)
  return result

# Функция для агрегирования данных за день в Google Sheets
def upload(cnt,cnt_success,cnt_users,mean_cnt):
    sh=client.create('Stats') # Всегда новая таблица
    sh.share('', perm_type='anyone', role='writer') # Таблица доступна для всех с ссылкой
    sh.sheet1.update(
        values=[['Дата', 'Всего попыток', 'Успешных попыток', 'Уникальных пользователей', 'Среднее количество успешных попыток пользователя']] +
               [[d, cnt[d], cnt_success[d], cnt_users[d], f"{mean_cnt[d]:.2f}"]
                for d in sorted(cnt)],
        range_name='A1'  # Явный именованный аргумент
    )
    sh_url=sh.url
    print(sh_url)
    return sh_url

# Класс для работы с БД test_db в Postgres в Colab
class Database:

  # Функция для соединения с БД
  def __init__(self, host, port, database, user, password, autocommit=False):
    self.connection=psycopg2.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password,    )
    if autocommit:
      self.connection.autocommit = True
    self.cursor = self.connection.cursor()
    logging.info("Подключение к БД установлено")

  # Фунция для вставки данных в БД
  def insert_data(self, data):
    if not data: return
    # Cоздание таблицы lti_data в БД test_db если ее нет
    self.cursor.execute("""
      CREATE TABLE IF NOT EXISTS lti_data (
      id SERIAL PRIMARY KEY,
      lti_user_id VARCHAR(255),
      oauth_consumer_key VARCHAR(255),
      lis_result_sourcedid TEXT,
      lis_outcome_service_url TEXT,
      is_correct INTEGER,
      attempt_type VARCHAR(50),
      created_at TIMESTAMP,
      loaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
      UNIQUE(lti_user_id, created_at)
      )
    """) # Поля таблицы
    for item in data:
      # Вставка значений в поля
      self.cursor.execute("""
        INSERT INTO lti_data
        (lti_user_id, oauth_consumer_key, lis_result_sourcedid,
         lis_outcome_service_url, is_correct, attempt_type, created_at)
         VALUES (%s, %s, %s, %s, %s, %s, %s)
         ON CONFLICT (lti_user_id, created_at)
         DO UPDATE SET loaded_at=CURRENT_TIMESTAMP """,
        (item['lti_user_id'], item['oauth_consumer_key'],
         item['lis_result_sourcedid'], item['lis_outcome_service_url'],
         item['is_correct'], item['attempt_type'], item['created_at']))
    self.connection.commit()
    logging.info(f"Загружено в БД {len(data)} записей")

  # Функция для закрытия соединения к БД
  def close(self):
    if hasattr(self, 'cursor') and self.cursor:
      self.cursor.close()
    if hasattr(self, 'connection') and self.connection:
      self.connection.close()
    logging.info("Подключение к БД закрыто")

data=get_post('https://b2b.itresume.ru/api/statistics',params={'client':'Skillfactory','client_key':'M2MGWS','start':'2023-04-01 12:46:47.860798','end':'2023-04-01 13:00:00.860798'})
if data:
  try:
    db = Database(DB_HOST, int(DB_PORT), DB_NAME, DB_USER, DB_PASSWORD)
    db.insert_data(data)
    db.close()
  except Exception as e:
    logging.error(f"Ошибка БД: {e}")




Всего попыток за день {'2023-04-01': 134}
Успешных попыток за день {'2023-04-01': 23}
Количество уникальных юзеров за день {'2023-04-01': 30}
Среднее количество решенных задач юзера за день {'2023-04-01': 0.77}
https://docs.google.com/spreadsheets/d/1PDTIWp4xc8XcuUSaePm-i-qtWEejWcGRJmIMjtJViZA


In [4]:
!apt-get update # Установка postgresql в Colab
!apt-get install -y postgresql postgresql-contrib
!service postgresql start

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,867 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,968 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,599 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [69.2 kB]
Get:14 http://arch

In [27]:
# Создание пользователя, пароля в Postgres и БД test_db в Colab
!sudo -u postgres psql -c "CREATE USER colab_user WITH PASSWORD 'colab_password';"
!sudo -u postgres psql -c "ALTER USER colab_user CREATEDB;"
!sudo -u postgres psql -c "CREATE DATABASE test_db OWNER colab_user;"


ERROR:  role "colab_user" already exists
ALTER ROLE
ERROR:  database "test_db" already exists


In [30]:
!sudo -u postgres psql -d test_db -c "SELECT * FROM lti_data LIMIT 10" > /content/output.txt 2>&1

In [26]:
!sudo -u postgres psql -d test_db -c "DROP TABLE IF EXISTS lti_data;"

NOTICE:  table "lti_data" does not exist, skipping
DROP TABLE
